In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score

import time

In [2]:
df = pd.read_csv('master_data_file.csv')

In [5]:
df.head()

,case_id,date_decision,MONTH,WEEK_NUM,target,actualdpd_943P,actualdpdtolerance_344P,addres_district_368M,addres_role_871L,addres_zip_823M,...,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,type_25L,typesuite_864L,validfrom_1069D
0,0,2019-01-03,201901,0,0,-1.0,-1,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,1,-1,0,1,-1,-1
1,1,2019-01-03,201901,0,0,-1.0,-1,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,1,-1,0,1,-1,-1
2,2,2019-01-04,201901,0,0,3.0,-1,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,1,-1,0,0,0,-1
3,3,2019-01-03,201901,0,0,3.0,-1,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,1,-1,0,1,0,-1
4,4,2019-01-04,201901,0,1,3.0,-1,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,1,-1,0,1,0,-1


In [6]:
df.shape

(1526659, 469)

In [7]:
df_week0 = df[df.WEEK_NUM ==0]

In [8]:
features = df.columns[5:]

In [9]:
X_0 = df_week0[features]
y_0 = df_week0.target

In [10]:
X_0_train, X_0_test, y_0_train, y_0_test = train_test_split(X_0, y_0, shuffle=True, random_state=123, test_size=0.1, stratify=y_0)

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [12]:
main_feature = []

for i in range(92):
    clf = DecisionTreeClassifier(max_depth=2, random_state=123)
    df_week = df[df.WEEK_NUM == i]
    X = df_week[features]
    y = df_week.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=123, test_size=0.1, stratify=y)
    clf.fit(X_train, y_train)
    main_feature.append(int(tree.export_text(clf).split(' ')[1].split('_')[1]))

In [13]:
dict_features = {}
for i in main_feature:
    dict_features[i] = 1 + dict_features.get(i,0)

In [14]:
dict_features

{243: 4,
 33: 35,
 41: 1,
 45: 1,
 361: 7,
 116: 1,
 117: 29,
 362: 2,
 262: 1,
 379: 1,
 149: 1,
 337: 1,
 81: 1,
 120: 3,
 367: 1,
 119: 1,
 123: 1,
 359: 1}

In [15]:
df_week.columns[33]

'assignmentdate_4527235D'

In [16]:
df_week.columns[117]

'currdebt_22A'

In [21]:
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),
                                n_estimators=200,
                                algorithm = 'SAMME',
                                learning_rate = 0.05,
                                random_state=123)

In [22]:
auc_list = []

for i in range(5):
    df_week = df[df.WEEK_NUM==i]

    X = df_week[features]
    y = df_week.target

    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=123, test_size=0.2, stratify=y)
    y2_train = 2*y_train - 1
    y2_test = 2*y_test - 1
    a = time.time()
    ada_clf.fit(X_train, y2_train)
    preds = ada_clf.predict_proba(X_test)
    score = roc_auc_score(y2_test, preds[:,1])
    b = time.time()
    print(f'Week: {i}, Score: {score}, Time: {round((b-a)/60,2)}')
    auc_list.append(score)

Week: 0, Score: 0.714367156412611, Time: 0.41
Week: 1, Score: 0.733839918946302, Time: 0.58
Week: 2, Score: 0.6855399568375784, Time: 0.53
Week: 3, Score: 0.7236031442532398, Time: 0.46
Week: 4, Score: 0.6960459200660567, Time: 0.41


In [25]:
df.shape

(1526659, 469)

In [26]:
df_week.shape

(14309, 469)

In [27]:
df2 = df[df.WEEK_NUM == 0]

In [42]:
X2_train, X2_test, y2_train, y2_test = train_test_split(df2[features], df2.target, shuffle=True, random_state=123, test_size=0.2, stratify=df2.target)

In [43]:
for i in range(1,92):
    df3 = df[df.WEEK_NUM == i]
    X3_train, X3_test, y3_train, y3_test = train_test_split(df3[features], df3.target, shuffle=True, random_state=123, test_size=0.2, stratify=df3.target)
    X2_train = pd.concat([X2_train, X3_train])
    X2_test = pd.concat([X2_test, X3_test])
    y2_train = pd.concat([y2_train, y3_train])
    y2_test = pd.concat([y2_test, y3_test])

In [48]:
X2_train.to_csv('X2_train.csv', index=False)

In [49]:
X2_test.to_csv('X2_test.csv', index=False)

In [50]:
y2_train.to_csv('y2_train.csv', index=False)

In [51]:
y2_test.to_csv('y2_test.csv', index=False)

In [55]:
#X3_train, X3_test, y3_train, y3_test = train_test_split(X2_train, y2_train, shuffle=True, random_state=123, test_size=0.5)
#y4_train = 2*y3_train - 1
#y4_test = 2*y3_test - 1
a = time.time()
ada_clf.fit(X3_train, y4_train)
preds = ada_clf.predict_proba(X3_train)
preds2 = ada_clf.predict_proba(X3_test)
score = roc_auc_score(y4_train, preds[:,1])
score2 = roc_auc_score(y4_test, preds2[:,1])
b = time.time()
print(f'Train Score: {score}, Test Score: {score2}, Time: {round((b-a)/60,2)}')

Train Score: 0.7765823634987499, Test Score: 0.7737864291999301, Time: 4.25


In [69]:
INDICES = X2_test.index

In [70]:
df_for_ada = df.loc[INDICES,:]

In [71]:
auc_list = []

for i in range(4):
    df_week = df_for_ada[df_for_ada.WEEK_NUM==i]

    X = df_week[features]
    y = df_week.target

    #X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=123, test_size=0.2, stratify=y)
    y2 = 2*y - 1
    #y2_test = 2*y_test - 1
    a = time.time()
    #ada_clf.fit(X, y2)
    preds = ada_clf.predict_proba(X)
    score = roc_auc_score(y2, preds[:,1])
    b = time.time()
    print(f'Week: {i}, Score: {score}, Time: {round((b-a)/60,2)}')
    auc_list.append(score)

Week: 0, Score: 0.7341655188246097, Time: 0.01
Week: 1, Score: 0.6282501085540599, Time: 0.01
Week: 2, Score: 0.6035455749038641, Time: 0.01
Week: 3, Score: 0.6191064220461926, Time: 0.01
